# Use of convolutions with tensorflow

In this notebook, you'll be using tensorflow to build a Convolutional Neural Network (CNN).  

### Convolution

Both, [this notebook](https://nbviewer.jupyter.org/github/marc-moreaux/Deep-Learning-classes/blob/master/notebooks/Convolution.ipynb) and this [wikipedia page](https://en.wikipedia.org/wiki/Convolution) might help you understand what is a convolution.

no, if we consider two functions $f$ and $g$ taking values from $\mathbb{Z} \to \mathbb{R}$ then:  
$ (f * g)[n] = \sum_{m = -\infty}^{+\infty} f[m] \cdot g[n - m] $

In our case, we consider the two vectors $x$ and $w$ :  
$ x = (x_1, x_2, ..., x_{n-1}, x_n) $  
$ w = (w_1, w_2) $

And get :   
$ x * w = (w_1 x_1 + w_2 x_2, w_1 x_2 + w_2 x_3, ..., w_1 x_{n-1} + w_2 x_n)$


#### Deep learning subtility :
    
In most of deep learning framewoks, you'll get to chose in between three paddings:
- **Same**: $(f*g)$ has the same shape as x (we pad the entry with zeros)
- **valid**: $(f*g)$ has the shape of x minus the shape of w plus 1 (no padding on x)
- **Causal**: $(f*g)(n_t)$ does not depend on any $(n_{t+1})$

### Tensorflow

"TensorFlow is an open-source software library for dataflow programming across a range of tasks. It is a symbolic math library, and also used for machine learning applications such as neural networks.[3] It is used for both research and production at Google often replacing its closed-source predecessor, DistBelief." - Wikipedia

We'll be using tensorflow to build the models we want to use. 

Here below, we build a AND gate with a very simple neural network :

In [7]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

# Define our Dataset
X = np.array([[0,0],[0,1],[1,0],[1,1]])
Y = np.array([0,0,0,1]).reshape(-1,1)

# Define the tensorflow tensors
x = tf.placeholder(tf.float32, [None, 2], name='X')  # inputs
y = tf.placeholder(tf.float32, [None, 1], name='Y')  # outputs
W = tf.Variable(tf.zeros([2, 1]), name='W')
b = tf.Variable(tf.zeros([1,]), name='b')

# Define the model
pred = tf.nn.sigmoid(tf.matmul(x, W) + b)  # Model

# Define the loss
with tf.name_scope("loss"):
    loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred) + (1-y) * tf.log(1-pred), reduction_indices=1))

# Define the optimizer method you want to use
with tf.name_scope("optimizer"):
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

# Include some Tensorboard visualization
writer_train = tf.summary.FileWriter("./my_model/")


# Start training session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer_train.add_graph(sess.graph)
    
    for epoch in range(1000):
        _, c, p = sess.run([optimizer, loss, pred], feed_dict={x: X,
                                                      y: Y})
print(p, y)

[[ 0.00839782]
 [ 0.1499088 ]
 [ 0.1499088 ]
 [ 0.78595555]] Tensor("Y:0", shape=(?, 1), dtype=float32)


To visualize the graph you just created, launch tensorbord.  
`$tensorboard --logdirs=./` on linux (with corresponding logdir)

---
### Get inspiration from the preceding code to build a XOR gate

Design a neural network with 2 layers.
- layer1 has 2 neurons (sigmoid or tanh activation)
- Layer2 has 1 neuron (it outouts the prediction)

And train  it

It's **mandatory** that you get a **tensorboard visualization** of your graph, try to make it look good, plz :)

Here below I put a graph of the model you want to have (yet your weights won't be the same)
![graph](https://i.stack.imgur.com/nRZ6z.png)

In [8]:
### Code here

tf.reset_default_graph()

# Define our Dataset
X = np.array([[0,0],[0,1],[1,0],[1,1]])
Y = np.array([0,1,1,0]).reshape(-1,1)

# Define the tensorflow tensors
x = tf.placeholder(tf.float32, [None, 2], name='X')  # inputs
y = tf.placeholder(tf.float32, [None, 1], name='Y')  # outputs
W1 = tf.Variable(tf.random_normal([2, 2]), name='W1')
b1 = tf.Variable(tf.random_normal([2,]), name='b1')
W2 = tf.Variable(tf.random_normal([2, 1]), name='W2')
b2 = tf.Variable(tf.random_normal([1,]), name='b2')

# Define the model
A2 = tf.nn.tanh(tf.matmul(x, W1) + b1)  # Model
pred = tf.nn.sigmoid(tf.matmul(A2, W2) + b2)

# Define the loss
with tf.name_scope("loss"):
    loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred) + (1-y) * tf.log(1-pred), reduction_indices=1))

# Define the optimizer method you want to use
with tf.name_scope("optimizer"):
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

# Include some Tensorboard visualization
writer_train = tf.summary.FileWriter("./my_modelXOR/")


# Start training session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer_train.add_graph(sess.graph)
    
    for epoch in range(10000):
        _, c, p = sess.run([optimizer, loss, pred], feed_dict={x: X,
                                                      y: Y})
    print_W1 = sess.run(tf.trainable_variables("W1"))
    print_W2 = sess.run(tf.trainable_variables("W2"))
print(p, y)

[[ 0.50119305]
 [ 0.49835789]
 [ 0.9979583 ]
 [ 0.00235485]] Tensor("Y:0", shape=(?, 1), dtype=float32)


### Print the weights of your model
And give an interpretation on what they are doing

In [9]:
### Code here
print("W1", print_W1)
print("W2", print_W2)

W1 [array([[-4.20842218,  4.61740065],
       [-1.56060135, -2.81463075]], dtype=float32)]
W2 [array([[ 4.02977324],
       [ 4.1637516 ]], dtype=float32)]


---
### Build a CNN to predict the MNIST digits
You can now move to CNNs. You'll have to train a convolutional neural network to predict the digits from MNIST.

You might want to reuse some pieces of code from [SNN](https://nbviewer.jupyter.org/github/marc-moreaux/Deep-Learning-classes/blob/master/notebooks/Intro_to_SNN.ipynb)

Your model should have 3 layers:
- 1st layer : 6 convolutional kernels with shape (3,3)
- 2nd layer : 6 convolutional kernels with shape (3,3)
- 3rd layer : Softmax layer

Train your model.

Explain all you do, and why, make it lovely to read, plz o:)

In [10]:
# Importation of data, instead of downloading via an external source, we use the data included in Tensorflow
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [11]:
# Helper Functions

def weight_variable(shape):
  initial = tf.random_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


In [12]:
# CNN


n_iteration = 20000
learning_rate = 0.01
batch_size = 50

tf.reset_default_graph()

# Initialize placeholders for input and output
with tf.variable_scope('Placeholders') as scope:
    x = tf.placeholder(tf.float32, [None, 784], name='X')  # inputs
    y = tf.placeholder(tf.float32, [None, 10], name='Y')  # outputs

# Define the first layer
with tf.variable_scope('Layer1') as scope:
    W_conv1 = weight_variable([3, 3, 1, 6])
    b_conv1 = bias_variable([6])
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    h_conv1 = conv2d(x_image, W_conv1) + b_conv1

# Define the second layer
with tf.variable_scope('Layer2') as scope:
    W_conv2 = weight_variable([3, 3, 6, 6])
    b_conv2 = bias_variable([6])
    h_conv2 = conv2d(h_conv1, W_conv2) + b_conv2

# Define the last layer
with tf.variable_scope('Layer3') as scope:
    # To flatten our tensor
    h_layer3 = tf.contrib.layers.flatten(h_conv2)
    logits = tf.layers.dense(h_layer3, 10)
    y_conv = tf.nn.softmax(logits)

# Loss
with tf.name_scope("loss"):
    entropy = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=y)
    loss = tf.reduce_mean(entropy)
    
# Optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

# Get infos on accuracy
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 

# To display the loss and the training accuracy in Tensorboard 
with tf.name_scope("summary"):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('loss', loss)
    merge_summary = tf.summary.merge_all()

# Print the logs for Tensorboard
writer_train = tf.summary.FileWriter("./my_modelMNIST/")

# Run session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer_train.add_graph(sess.graph)
    
    for i in range(n_iteration):
        total_loss = 0
        batch = mnist.train.next_batch(batch_size)
        _, l, summary= sess.run([optimizer, loss, merge_summary], feed_dict={x: batch[0], y: batch[1]})
        total_loss += l
        if i % 100 == 0:
          train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1]})
          print('step :', i, 'training accuracy :', train_accuracy, 'loss :', total_loss/(len(batch[0])))
        writer_train.add_summary(summary, i)

    print_W_conv1 = sess.run(W_conv1)
    print_W_conv2 = sess.run(W_conv2)
    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))


step : 0 training accuracy : 0.2 loss : 0.0461328697205
step : 100 training accuracy : 0.9 loss : 0.00744406580925
step : 200 training accuracy : 0.92 loss : 0.0075448256731
step : 300 training accuracy : 0.88 loss : 0.00656681001186
step : 400 training accuracy : 0.94 loss : 0.00524492442608
step : 500 training accuracy : 0.88 loss : 0.00783619523048
step : 600 training accuracy : 0.9 loss : 0.00688884556293
step : 700 training accuracy : 0.88 loss : 0.00921800553799
step : 800 training accuracy : 0.96 loss : 0.00392190784216
step : 900 training accuracy : 0.98 loss : 0.00367008507252
step : 1000 training accuracy : 0.86 loss : 0.0116500866413
step : 1100 training accuracy : 0.86 loss : 0.0114709615707
step : 1200 training accuracy : 0.92 loss : 0.00841698467731
step : 1300 training accuracy : 0.98 loss : 0.00550443410873
step : 1400 training accuracy : 0.92 loss : 0.0100123286247
step : 1500 training accuracy : 0.92 loss : 0.0081374257803
step : 1600 training accuracy : 0.94 loss : 0

step : 13500 training accuracy : 0.9 loss : 0.00609301447868
step : 13600 training accuracy : 0.88 loss : 0.00821358025074
step : 13700 training accuracy : 0.94 loss : 0.00307634145021
step : 13800 training accuracy : 0.92 loss : 0.00693317055702
step : 13900 training accuracy : 0.9 loss : 0.00895529270172
step : 14000 training accuracy : 0.94 loss : 0.00415851861238
step : 14100 training accuracy : 0.9 loss : 0.004808421731
step : 14200 training accuracy : 0.94 loss : 0.00363384157419
step : 14300 training accuracy : 0.96 loss : 0.00303963065147
step : 14400 training accuracy : 0.86 loss : 0.00863587617874
step : 14500 training accuracy : 0.88 loss : 0.00621112048626
step : 14600 training accuracy : 0.96 loss : 0.00376538097858
step : 14700 training accuracy : 0.94 loss : 0.00535335242748
step : 14800 training accuracy : 0.9 loss : 0.00940073788166
step : 14900 training accuracy : 0.96 loss : 0.00389783322811
step : 15000 training accuracy : 0.96 loss : 0.00342514634132
step : 15100 t

### Print the weights of your model
And give an interpretation on what they are doing

In [13]:
print("W_con1", print_W_conv1)
print("W_con2", print_W_conv2)

W_con1 [[[[ 0.0339084  -0.00574003  0.09405904  0.01899241  0.02146713  0.03116777]]

  [[-0.01389475 -0.02093893 -0.04680197 -0.06296932 -0.0403694  -0.0970225 ]]

  [[-0.02533246  0.04760861  0.00739993  0.08743949  0.01397227  0.06249897]]]


 [[[-0.05389746  0.03409469 -0.03007606  0.00130187 -0.01885582  0.0033858 ]]

  [[-0.00608744 -0.02461667  0.36268565 -0.00600112  0.03524905  0.06681167]]

  [[ 0.04017597 -0.02518635  0.04381526 -0.0172054  -0.02788767 -0.02161105]]]


 [[[ 0.00343605 -0.04836487 -0.04212236  0.01002444  0.0163784  -0.00384359]]

  [[ 0.00457347  0.04254678  0.02201492  0.01601308 -0.06910476 -0.03424655]]

  [[-0.04937457 -0.02119005 -0.03981137 -0.00272664  0.03052462  0.07071938]]]]
W_con2 [[[[  6.12297431e-02   9.22521055e-02   1.02953715e-02   7.01771379e-02
      7.84876756e-03   9.97670963e-02]
   [ -4.74824458e-02   3.09986360e-02  -1.93826538e-02  -2.45424956e-02
      7.28035420e-02   6.10135645e-02]
   [ -2.01146472e-02   1.55939564e-01  -7.651229

### Chose one (tell me what you chose...)
- Show how the gradients (show only one kernel) evolve for good and wrong prediction. (hard)
- Initialize the kernels with values that make sense for you and show how they evolve. (easy) 
- When training is finished, show the 6+6=12 results of some convolved immages. (easy)

In [14]:
### Code here